# Projekt Code Induktive Erwärmung

In [1]:
from ngsolve import *
from netgen.geom2d import unit_square
from ngsolve.webgui import Draw
from netgen.csg import *
from netgen.geom2d import SplineGeometry
import math
import numpy as np

### Konstanten

In [2]:
omega = 2*pi*50     # Frequenz 50 Hz
mu0 = 4*pi*1e-7     # Magnetische Feldkonstante

sigmaCu = 56e6      # S/m (Kupfer)
sigmaCore = 56e6    # S/m (Kupfer)

# Reihenfolge: air, copper, core
sigma = CoefficientFunction([0 , sigmaCu , sigmaCore ])
mur = CoefficientFunction([1 ,1 ,1])
nu = CoefficientFunction(1/mu0 * 1/mur ) # Permeabilität des Materials

ic = 10             # Strom in den Windungen (A)

# Mesh Generation
l_air = 0.1         # Grosse der dargestellten Luft um denn Versuchsaufbau
r_wire = 0.001      # Drahtdurchmesser in Metern
nx_wire = 10        # Anzahl Kabel in X-Richtung
ny_wire = 3         # Anzahl Kabel in Y-Richtung
dxdy_wire = 0.5*r_wire

rA_core = 0.01      # Core Höhe
rI_core = 0.008
l_core = 0.08       # Core Länge
epsZero = 1e-8

### Mesh generation

In [3]:
geo = SplineGeometry()
geo.AddRectangle(p1=(-l_air/2,epsZero),
                 p2=(l_air/2,l_air/2),
                 bcs=["rotsym","outer","outer","outer"],
                 leftdomain=1,
                 rightdomain=0)
pts = []
for i in range(nx_wire):
    for j in range(ny_wire):
        pts.append([
            i*(2*r_wire+dxdy_wire), 
            rA_core+dxdy_wire+r_wire+j*(2*r_wire+dxdy_wire)
        ])
        geo.AddCircle(
            c=(i*(2*r_wire+dxdy_wire), 
               rA_core+dxdy_wire+r_wire+j*(2*r_wire+dxdy_wire)),
            r=r_wire, bc="inner",
            leftdomain=2,
            rightdomain=1)
pts = np.array(pts)
geo.AddRectangle(p1=(-l_core/2,rI_core),
                 p2=(l_core/2,rA_core),
                 bc="inner",
                 leftdomain=3,
                 rightdomain=1)
geo.SetMaterial (1, "air")
geo.SetMaterial (2, "copper")
geo.SetMaterial (3, "core")
geo.SetDomainMaxH(3,(rA_core-rI_core)/2)

mesh = Mesh(geo.GenerateMesh(maxh=0.0025))

# Visualisierung
mat_mapping = {name: i+1 for i, name in enumerate(mesh.GetMaterials())}
cf = mesh.MaterialCF(mat_mapping)
Draw(cf, mesh, "material_colors")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

## Aufgabe 1
1) Wie lautet die schwache Gleichung für die partielle Differentialgleichung (4)? Verifizieren Sie diese am Listing 2.
2) Berechnen Sie die Lösung und stellen Sie diese graphisch dar. Nutze Sie für die
Berechnung die Formel (5).
3) Berechnen Sie die induktive Wärmequelle q mit Hilfe von Formel (7).

#### 1) Schwache Gleichung der partiellen Differentialgleichung

Die Differentialgleichung lautet:
$$
\sigma j \omega A_\phi - \left( \frac{\partial}{\partial r} \left( \frac{\mu^{-1}}{r} \frac{\partial (r A_\phi)}{\partial r} \right) + \frac{\partial}{\partial z} \left( \mu^{-1} \frac{\partial A_\phi}{\partial z} \right) \right) = j_\phi
$$

Die schwache Formulierung ergibt sich, indem man beide Seiten mit einer Testfunktion  $v \in H^1$ (komplexwertiger FE-Raum) multipliziert und über das Volumen  $\Omega$ integriert:

$$
\int_\Omega \left( \sigma j \omega A_\phi v - \frac{\partial}{\partial r} \left( \frac{\mu^{-1}}{r} \frac{\partial (r A_\phi)}{\partial r} \right) v - \frac{\partial}{\partial z} \left( \mu^{-1} \frac{\partial A_\phi}{\partial z} \right) v \right) \, d\Omega = \int_\Omega j_\phi v \, d\Omega
$$

Das Teilintegral mit zwei Ableitungen nach $r$ lässt sich u.A. durch partielle Integration vereinfachen.
$$ \frac{\partial (r A_\phi)}{\partial r}=A_\phi + r \frac{\partial (A_\phi)}{\partial r} \quad (Produktregel)$$
$$
\int_\Omega - \frac{\partial}{\partial r} \left( \frac{\mu^{-1}}{r} \frac{\partial (r A_\phi)}{\partial r} \right) v \, d\Omega = \underbrace{ \left[ - \frac{\mu^{-1}}{r} \frac{\partial (r A_\phi)}{\partial r} v \right]_{\partial\Omega}}_{=0} +
\int_\Omega \frac{\mu^{-1}}{r} \frac{\partial (r A_\phi)}{\partial r} \frac{\partial v}{\partial r} \, d\Omega =  \int_\Omega \mu^{-1} \left( \frac{A_\phi}{r} +  \frac{\partial A_\phi}{\partial r} \right)  \frac{\partial v}{\partial r} \, d\Omega 
$$

Die partielle Integration wird auch aufs Teilintegral mit zwei Ableitungen nach $z$ angewendet.
$$
\int_\Omega - \frac{\partial}{\partial z} \left( \mu^{-1} \frac{\partial A_\phi}{\partial z} \right) v \, d\Omega = \underbrace{ \left[ - \mu^{-1} \frac{\partial A_\phi}{\partial z} v \right]_{\partial\Omega}}_{=0} +
\int_\Omega \mu^{-1} \frac{\partial A_\phi}{\partial z} \frac{\partial v}{\partial z} \, d\Omega
$$

Die schwache Formulierung der Differentialgleichung lautet:

> $$ \underbrace{ \int_\Omega \sigma j \omega A_\phi v  + \mu^{-1} \left( \frac{A_\phi}{r} +  \frac{\partial A_\phi}{\partial r} \right)  \frac{\partial v}{\partial r} + \mu^{-1} \frac{\partial A_\phi}{\partial z} \frac{\partial v}{\partial z} \, d\Omega }_{\text{Bilinearform } a(A_\phi, v)} = \underbrace{ \int_\Omega j_\phi v \, d\Omega }_{\text{Linearfrom }  f(v)} $$

wobei die Bilinearform  $a(A_\phi, v)$  durch die folgenden Terme definiert ist:

$$
a(A_\phi, v) = \int_\Omega \sigma j \omega A_\phi v \, d\Omega + \int_\Omega \mu^{-1} \left( \frac{A_\phi}{r} +  \frac{\partial A_\phi}{\partial r} \right)  \frac{\partial v}{\partial r} \, d\Omega  + \int_\Omega \mu^{-1} \frac{\partial A_\phi}{\partial z} \frac{\partial v}{\partial z} \, d\Omega
$$

#### 2) Numerische Lösung
##### Berechnung von $A_\phi$

In [4]:
### Listing 1 : Definition FE-Raum für Wirbelstromproblem
V = H1(mesh,order=3, complex = True, dirichlet='rotsym')
u,v = V.TnT()
gfu = GridFunction(V)

In [5]:
### Listing 2 : Definition Wirbelstrom Problem
# Wechsel in Zylinderkoordinaten (es wird nur ein Querschnitt dargestellt (phi = 0))
r = y
z = x
uz, ur = grad(u)
vz, vr = grad(v) 

# Schwache Gleichung als Bilinearform
a = BilinearForm(V)
a += (nu*(1/r*u+ur)*vr + nu*uz*vz)*dx # erster Bilinear term
a += 1j*omega*sigma*u*v*dx(definedon=mesh.Materials('copper|core')) # zweiter Bilinear term (sigma=0 in Luft)

Jimp = CoefficientFunction([0,ic/(r_wire**2*pi),0]) # Richtung und Stärke des Stroms al Impuls angegeben

f = LinearForm(V)
f += Jimp*v*dx(definedon=mesh.Materials('copper'))

a.Assemble()
f.Assemble()

gfu.vec.data = a.mat.Inverse(V.FreeDofs()) * f.vec

print("A_phi (Norm)")
Draw(Norm(gfu), mesh, "A_phi")

A_phi (Norm)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

##### Berechnung von $B_r$ und $B_z$
$$ B_r = - \frac{\partial A_\phi}{\partial z}$$

$$ B_z = \frac{1}{r} \frac{\partial (r A_\phi)}{\partial r} = \frac{A_\phi}{r} + \frac{\partial (A_\phi)}{\partial r}$$

In [6]:
# Create the coefficient functions
grad_gfu = grad(gfu)
B_r = -grad_gfu[0]  # Negative derivative with respect to z (x-coordinate)
#B_z = gfu / r + grad_gfu[1]  # Derivative with respect to r (y-coordinate)
rAphi = GridFunction(V)
rAphi.Set(r * gfu)
B_z = grad(rAphi)[1] / r
B_z = IfPos(r-epsZero, B_z, 0) # Handle potential division by zero at r=0 axis

# Create GridFunctions for visualization
B_r_gf = GridFunction(V)
B_z_gf = GridFunction(V)
# Project the expressions onto the finite element space
B_r_gf.Set(B_r)
B_z_gf.Set(B_z)

# Visualize Components
print("B_z:")
Draw(Norm(B_z_gf), mesh, "B_z")
print("B_r:")
Draw(Norm(B_r_gf), mesh, "B_r")

# Visualize as vector field
B_field = CoefficientFunction((Norm(B_z_gf), Norm(B_r_gf)))  # For 2D visualization
print("B = (B_z, B_r) (Vector):")
Draw(B_field, mesh, "B (Vector Field)", vectors=True)

print("B (Norm):")
Draw(Norm(B_field), mesh, "B (complex)", vectors=True)


B_z:


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

B_r:


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

B = (B_z, B_r) (Vector):


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

NameError: name 'Vreal' is not defined

#### 2 Induktive Wärmequelle
Die Formel (7) für die induktive Wärmequelle lautet:

$$
q = \frac{1}{2} | \mathbf{J} \cdot \mathbf{E} | = \frac{\sigma}{2} |\mathbf{E}|^2
$$

Dabei gilt:

$\sigma$ ist die elektrische Leitfähigkeit (z. B. $56 \times 10^6 \, \text{S/m}$ für Kupfer)

$\mathbf{E}$ ist das elektrische Feld

$$
\mathbf{J} = \sigma \mathbf{E} \quad \text{(Ohmsches Gesetz für Leiter)}
$$

Die Größe $q$ beschreibt die Wärmeerzeugung pro Volumen $(\text{W/m}^3)$ aufgrund des induktiven Joule-Effekts.

Im rotationssymmetrischen Fall ist das elektrische Feld gegeben durch:

$$
\mathbf{E} = -j \omega A_\phi \quad \text{(laut Formel (6))}
$$

Da wir das Quadrat des Betrags benötigen:

$$
|\mathbf{E}|^2 = |\omega A_\phi|^2 = \omega^2 |A_\phi|^2
$$

Daraus ergibt sich die Wärmequelle $q$:

$$
q = \frac{\sigma}{2} \omega^2 |A_\phi|^2
$$

> stimmt so nicht, weil J = Jexp + J int

In [ ]:
### Listing 3 : Berechnung der induktiven Wärmequelle
Ez = -1j*omega*gfu  # Elektrische Feldstärke
Jz = sigma*Ez       # Elektrischer Strom
Jtot = Jz + Jimp
Qe = 1/2*Norm(InnerProduct(Jtot,Conj(Ez))) # Induktive Wärmequelle
#print(Qe)
Draw(Qe, mesh)

Q2 = 1/2 * sigma * Norm(gfu)**1
#Q2 = 1/2 * Norm(sigma * InnerProduct(Ez, Conj(Ez)) + Jimp*Ez)
Draw(Q2, mesh)

Q3 = 1/2 * Norm( sigma * omega**2 * gfu*Conj(gfu) - 1j * omega * Jimp * gfu)

Draw(Qe-Q2, mesh)
Draw(Qe-Q3, mesh)

